In [4]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import Session
from sqlalchemy import MetaData
from sqlalchemy import ForeignKey
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import insert
from sqlalchemy import select
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer,Float, String, MetaData
from sqlalchemy.orm import Session


In [5]:
engine = create_engine("sqlite+pysqlite:///C://Study//CSE 4000//New folder//Thesis//dataset//New folder//chicago.crime.db", echo=False)


In [7]:
# Load data using Pandas (replace 'your_data.csv' with your dataset file)
data = pd.read_csv('C://Study//CSE 4000//New folder//Thesis//dataset//Crimes_-_2001_to_Present.csv')

# Get the column names and data types from the CSV
column_names = data.columns
column_data_types = {col: data[col].dtype for col in column_names}

# Create a new MetaData object and a users table based on the column names and data types
metadata = MetaData()

CrimeChicagoTable = Table(
    'CrimeChicagoTable',
    metadata,
    Column('id', Integer, primary_key=True),
    *[Column(name, String) if dtype == 'object' else
      Column(name, Integer) if dtype == 'int64' else
      Column(name, Float) for name, dtype in column_data_types.items()]
)

# Create the table in the database
metadata.create_all(engine)

# Start a session
session = Session(engine)

# Loop through the data and insert into the table, skipping rows with null values
for _, row in data.iterrows():
    datarow = {name: row[name] for name in column_names if not pd.isna(row[name])}
    session.execute(CrimeChicagoTable.insert().values(datarow))
    session.commit()


OperationalError: (sqlite3.OperationalError) duplicate column name: ID
[SQL: 
CREATE TABLE "CrimeChicagoTable" (
	id INTEGER NOT NULL, 
	"ID" INTEGER, 
	"Case Number" VARCHAR, 
	"Date" VARCHAR, 
	"Block" VARCHAR, 
	"IUCR" VARCHAR, 
	"Primary Type" VARCHAR, 
	"Description" VARCHAR, 
	"Location Description" VARCHAR, 
	"Arrest" FLOAT, 
	"Domestic" FLOAT, 
	"Beat" INTEGER, 
	"District" FLOAT, 
	"Ward" FLOAT, 
	"Community Area" FLOAT, 
	"FBI Code" VARCHAR, 
	"X Coordinate" FLOAT, 
	"Y Coordinate" FLOAT, 
	"Year" INTEGER, 
	"Updated On" VARCHAR, 
	"Latitude" FLOAT, 
	"Longitude" FLOAT, 
	"Location" VARCHAR, 
	PRIMARY KEY (id)
)

]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# SELECT the first few rows with column names
query = CrimeChicagoTable.select().limit(5)  # You can adjust the limit as needed
result = session.execute(query)

# Get the column names and the data
column_names = result.keys()
data = result.fetchall()

# Print the column names
print(column_names)

# Print the first few rows of data
for row in data:
    print(row)

# Close the session
#session.close()

# Commit changes
session.commit()